# 02_parse_raw.ipynb

## 1. Objective
Exploratory notebook to inspect the raw Google Shopping JSON and design the staging schema.

**Goals**

- Pull a small sample from `raycon.raw_google_shopping`
- Understand the structure of `response_json` (top-level sections and nested lists)
- Draft the structure of staging tables (`stg_google_shopping_searches`, `stg_google_shopping_results`)
- This notebook is **read-only** against the database (no writes).

## 2. Setup and Configuration

### 2.1 Imports

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('future.no_silent_downcasting', True)
import json

from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

### 2.2 Connect to Database

In [2]:
load_dotenv()

user = os.getenv('PGUSER')
password = os.getenv('PGPASSWORD')
pghost = os.getenv('PGHOST')
pgport = os.getenv('PGPORT')
pgdatabase = os.getenv('PGDATABASE')

engine = create_engine(
    f'postgresql+psycopg2://{user}:{password}@{pghost}:{pgport}/{pgdatabase}')

# Verify integrity of connection
pd.read_sql("SELECT * FROM raycon.raw_google_shopping ORDER BY id DESC LIMIT 3", engine)

,id,pulled_at,keyword,page,response_json
0,147,2025-12-19 16:30:55.233203+00:00,bluetooth headphones,1,"{'filters': [{'type': 'Refine results', 'optio..."
1,146,2025-12-19 16:30:45.132971+00:00,wireless earbuds,1,"{'filters': [{'type': 'Refine results', 'optio..."
2,145,2025-12-19 16:30:41.305642+00:00,wireless headphones,1,"{'filters': [{'type': 'Refine results', 'optio..."


## 3. Pull Sample Rows
Grab a small sample of the most recent raw rows so we can inspect the
`response_json` structure.

In [3]:
# One row = one SerpAPI call for a given keyword + page.
query = '''
SELECT id, pulled_at, keyword, page, response_json
FROM raycon.raw_google_shopping
ORDER BY pulled_at DESC
LIMIT 7;
'''

with engine.connect() as conn:
    df_raw = pd.read_sql(query, conn)

In [4]:
df_raw

,id,pulled_at,keyword,page,response_json
0,147,2025-12-19 16:30:55.233203+00:00,bluetooth headphones,1,"{'filters': [{'type': 'Refine results', 'optio..."
1,146,2025-12-19 16:30:45.132971+00:00,wireless earbuds,1,"{'filters': [{'type': 'Refine results', 'optio..."
2,145,2025-12-19 16:30:41.305642+00:00,wireless headphones,1,"{'filters': [{'type': 'Refine results', 'optio..."
3,144,2025-12-19 16:30:31.428345+00:00,best earbuds,1,"{'filters': [{'type': 'Refine results', 'optio..."
4,143,2025-12-19 16:30:26.151771+00:00,best headphones,1,"{'filters': [{'type': 'Refine results', 'optio..."
5,142,2025-12-19 16:30:23.082352+00:00,earbuds,1,"{'filters': [{'type': 'Refine results', 'optio..."
6,141,2025-12-19 16:30:15.913642+00:00,headphones,1,"{'filters': [{'type': 'Refine results', 'optio..."


## 4. Inspect JSON
Take one `response_json` payload from the sample and inspect:
- Top-level keys (major sections of the API response)
- Truncated view of the full nested structure

This helps decide which parts deserve staging tables and which parts are to be left out.

### 4.1 Extract one JSON Response

In [15]:
sample = df_raw.loc[6, :]
id, pulled_at, keyword, page = sample.loc[['id', 'pulled_at', 'keyword', 'page']]

sample = sample.loc['response_json']
list(sample.keys())

['filters',
 'search_metadata',
 'shopping_results',
 'search_parameters',
 'search_information',
 'serpapi_pagination',
 'categorized_shopping_results']

### 4.2 Preview for Each Key

In [6]:
# Loop through each top-level key and preview the structure of its value
for key in sample.keys():
    print(f"======== {key} ========")
    try:
        # Print only the first part of each key’s value
        print(json.dumps(sample[key], indent=2)[:5000])
    except TypeError:
        print(sample[key])
    print("\n\n\n\n")

======== filters ========
[
  {
    "type": "Refine results",
    "options": [
      {
        "text": "On sale",
        "shoprs": "CAESBEoCGAEYBioUYmx1ZXRvb3RoIGhlYWRwaG9uZXMyEwgGEgdPbiBzYWxlGAIiBEoCGAFY6bcgYAI",
        "serpapi_link": "https://serpapi.com/search.json?device=desktop&engine=google_shopping&gl=us&google_domain=google.com&hl=en&location=Los+Angeles%2CCalifornia%2CUnited+States&num=100&q=bluetooth+headphones+sale&shoprs=CAESBEoCGAEYBioUYmx1ZXRvb3RoIGhlYWRwaG9uZXMyEwgGEgdPbiBzYWxlGAIiBEoCGAFY6bcgYAI"
      },
      {
        "text": "Nearby",
        "shoprs": "CAEYAyoUYmx1ZXRvb3RoIGhlYWRwaG9uZXMyDAgDEgZOZWFyYnkYAljptyBgAg",
        "serpapi_link": "https://serpapi.com/search.json?device=desktop&engine=google_shopping&gl=us&google_domain=google.com&hl=en&location=Los+Angeles%2CCalifornia%2CUnited+States&num=100&q=bluetooth+headphones+nearby&shoprs=CAEYAyoUYmx1ZXRvb3RoIGhlYWRwaG9uZXMyDAgDEgZOZWFyYnkYAljptyBgAg"
      },
      {
        "text": "Get it tomorrow",
        "

## 5. Document Structure and Entities
From inspecting the JSON payload, the response contains several top-level sections.  
Only some contain business-relevant data worth staging.

### Likely Irrelevant (meta / API mechanics)
- `filters` – UI refinements unrelated to product data  
- `search_metadata` – timestamps, request IDs, URLs  
- `search_information` – echo of the query text  
- `serpapi_pagination` – next-page links

### Relevant Entities
#### **1. Keyword Search**

Represents a single keyword search event.
Comes primarily from:
- `search_parameters`

Each raw row → one search event (one keyword, one page).

#### **2. Product Results**
Represents a product appearing in search results.
Found in:
- `shopping_results`
- `categorized_shopping_results` (same structure but grouped into categories)

These two lists can be unified into a single entity with a `category` field (`NULL` or `"general"` for uncategorized results).

## 6. Determine Staging Tables
Based on the entities:

### 6.1 stg_searches
One row per keyword search. \
Fields sourced from `search_parameters` plus:
- `raw_id` (FK to raw table)
- `pulled_at`
- `keyword`
- `page`


### 6.2 stg_results
One row per product result.   

**Core fields:**
- stg_result_id
- tag
- title
- extracted_price & extracted_old_price
- rating
- reviews
- source
- multiple_sources
- product_id

**Module fields:**
- module_type
- module_label
- module_index
- block_position
- position_in_module

**Search context:**
- raw_id
- keyword
- page
- pulled_at

This table will contain ~50-75 rows per SerpAPI pull request.

## 7. Prototype `stg_searches` Flattening Logic
The goal of this section is to prototype how raw search metadata from the SerpAPI
response will be transformed into a clean `stg_searches` record.

This table represents **one row per keyword search** and contains:
- Identifiers linking back to `raw_google_shopping` (`raw_id`, `pulled_at`, `keyword`, `page`)
- Search parameters extracted from the `"search_parameters"` object
- Only values actually useful for analytics or debugging (e.g., device, number of results requested)

Once validated here, this logic will later be moved into the proper staging notebook
where it will run for **every raw record** during ingestion.

In [7]:
def build_searches_for_keyword(keyword_raw):
    """
    Build the stg_searches record for a single raw search event.

    This function:
    - Takes the first row of df_raw (prototype mode)
    - Extracts identifiers (raw_id, pulled_at, keyword, page)
    - Extracts search parameters from response_json["search_parameters"]
    - Places everything into a clean, single-row DataFrame

    Returns:
        pd.DataFrame: one clean stg_search row
    """
    
     # Grab the first raw record (example row in this case)
    search_row = keyword_raw

     # Pull identifiers & search metadata from the raw table
    raw_id, pulled_at, keyword, page = search_row[['id', 'pulled_at', 'keyword', 'page']]
    
    # Extract the "search_parameters" object from the JSON
    params = search_row["response_json"]["search_parameters"]

    # Build the clean staging DataFrame
    search_df_clean = pd.DataFrame([{
    "raw_id": raw_id,
    "pulled_at": pulled_at,
    "keyword": keyword,
    "page": page,
    "location_used": params.get("location_used"),
    "location_requested": params.get("location_requested"),
    "gl": params.get("gl"),
    "hl": params.get("hl"),
    "device": params.get("device"),
    "num_results_requested": int(params.get("num")),
    "engine": params.get("engine"),
    "google_domain": params.get("google_domain"),
    }])
    return search_df_clean
    
build_searches_for_keyword(df_raw.loc[2, :])

,raw_id,pulled_at,keyword,page,location_used,location_requested,gl,hl,device,num_results_requested,engine,google_domain
0,145,2025-12-19 16:30:41.305642+00:00,wireless headphones,1,"Los Angeles,California,United States","Los Angeles,California,United States",us,en,desktop,100,google_shopping,google.com


## 8. Prototype stg_results Flattening Logic

Goal: take a single raw SerpAPI response and turn all product rows
(for that keyword+page) into a clean, tabular DataFrame that matches
our upcoming `stg_results` schema.

We do this in two layers:

1. **`transform_results_df`**  
   Takes a raw “results” DataFrame for one module (either 'All products'/uncategorized, inline
   or a specific category), drops/renames columns, normalizes types, and
   adds module + search context fields.

2. **`build_results_for_keyword`**  
   For one raw row in `raycon.raw_google_shopping`:
   - Flatten `shopping_results` into the “All products” category
   - Run 'All products' piece through `transform_results_df` 
   - Flatten each entry in `categorized_shopping_results` (if they exist) into their respective categories
   - Flatten `inline_shopping_results` (if it exists for a given keyword search)
   - Run each piece through `transform_results_df`  
   - Union 'All products' & the potential categorized results & inline results into one DataFrame: **one row per product result**.

In [8]:
def transform_results_df(
    df_in,
    *,
    raw_id,
    keyword,
    page,
    pulled_at,
    module_type,
    module_label,
    module_index):
    """
    Take a raw shopping-results DataFrame for a single module
    and return a cleaned, standardized results DataFrame.

    - Drops unused SerpAPI fields
    - Normalizes `multiple_sources` to boolean
    - Renames extracted price fields
    - Adds module + search context columns
    - Reorders columns to match `stg_results` schema
    """
    # Work on a copy to avoid mutating the original input
    transform_df = df_in.copy()

    # 1) Drop columns we decided not to stage
    transform_df = transform_df.drop(columns=['snippet', 'price', 'delivery', 'old_price', 'thumbnail',
                                             'extensions', 'source_icon', 'product_link', 'serpapi_thumbnail', 
                                            'immersive_product_page_token', 'serpapi_immersive_product_api'],
                                    errors='ignore')

    # 2) Convert multiple_sources -> proper bool
    if 'multiple_sources' in transform_df:
        transform_df['multiple_sources'] = (
            transform_df['multiple_sources'].replace('True', True).fillna(False).astype(bool))
    else:
        transform_df['multiple_sources'] = False

    # 3) Create and nullify block_position for results which lack this field
    transform_df["block_position"] = transform_df.get("block_position", None)

    # 4) Rename fields to final names
    transform_df = transform_df.rename(columns={
        'position': 'position_in_module',
        'extracted_price': 'price',
        'extracted_old_price': 'old_price'
    })

    # 5) Attach module + search context
    transform_df = transform_df.assign(
        module_type=module_type,
        module_label=module_label,
        module_index=module_index,
        raw_id=raw_id,
        keyword=keyword,
        page=page,
        pulled_at=pulled_at
    )

    # 6) Reorder columns to match target table
    final_cols = [
    "raw_id",
    "keyword",
    "page",
    "pulled_at",
    "title",
    "product_id",
    "price",
    "old_price",
    "reviews",
    "rating",
    "source",
    "multiple_sources",
    "tag",
    "module_type",
    "module_label",
    "module_index",
    "block_position",
    "position_in_module"
    ]
    transform_df = transform_df.reindex(columns=final_cols)
    
    return transform_df

In [17]:
def build_results_for_keyword(keyword_raw):
    """
    Build a full results DataFrame for ONE raw keyword request row.

    Steps:
    - Pull raw_id / keyword / page / pulled_at from df_raw
    - Flatten `shopping_results` (all products)
    - Flatten each entry in `categorized_shopping_results`
    - Run everything through `transform_results_df`
    - Union uncategorized + categorized into one DataFrame
    """
     # 1) Grab the first raw row (later this will be parameterized / looped)
    keyword_search = keyword_raw

     # 2) Extract search metadata used for context columns
    raw_id, pulled_at, keyword, page = keyword_search.loc[['id', 'pulled_at', 'keyword', 'page']]

    # 3) Extract JSON extract from the keyword request
    keyword_json = keyword_search.loc['response_json']

    # 4) Create a blank dataframe list
    df_list = []

    # 5) Build uncategorized results
    uncategorized_df_raw = pd.DataFrame(keyword_json['shopping_results'])
    uncategorized_df_clean = transform_results_df(uncategorized_df_raw, 
                                                raw_id=raw_id, pulled_at=pulled_at, keyword=keyword, page=page,
                                                module_type='all_products', module_label='All products', module_index=99)
    
    df_list.append(uncategorized_df_clean)

    # 6) (If they exist) Loop over categorized modules, flatten + transform each
    categorized_df_raw = pd.DataFrame(keyword_json.get('categorized_shopping_results', []))
    if categorized_df_raw.empty:
        categorized_df_clean = categorized_df_raw
    else:
        for i in range(len(categorized_df_raw)):
            category_title = categorized_df_raw['title'][i]
            per_category_results_raw = pd.json_normalize(categorized_df_raw['shopping_results'][i])
            per_category_results_clean = transform_results_df(per_category_results_raw,
                                                             raw_id=raw_id, pulled_at=pulled_at, keyword=keyword, page=page,
                                                             module_type='categorized_products', module_label=categorized_df_raw['title'][i], module_index=i+1)
            if i == 0:
                categorized_df_clean = per_category_results_clean.copy()
            else:
                categorized_df_clean = (pd.concat([categorized_df_clean, per_category_results_clean], 
                                            ignore_index=True))
            
    df_list.append(categorized_df_clean)

    #7) (If they exist) Build inline results
    inline_df_raw = pd.DataFrame(keyword_json.get('inline_shopping_results', []))
    if inline_df_raw.empty:
                inline_df_clean = inline_df_raw
    else:
        inline_df_clean = transform_results_df(inline_df_raw, 
                                                raw_id=raw_id, pulled_at=pulled_at, keyword=keyword, page=page,
                                                module_type='inline_products', module_label='Inline products', module_index=100)
    
    df_list.append(inline_df_clean)

    # 8) Return union of uncategorized + categorized (if exists) + inline (if exists) into one df
    return pd.concat(df_list, ignore_index=True)
    
build_results_for_keyword(df_raw.loc[1, :])

,raw_id,keyword,page,pulled_at,title,product_id,price,old_price,reviews,rating,source,multiple_sources,tag,module_type,module_label,module_index,block_position,position_in_module
0,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,JBL Tune 245 NC,3393824327707612313,39.95,110.0,278.0,4.3,Best Buy,True,63% OFF,all_products,All products,99,None,1
1,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,Soundcore P40i by Anker Noise Cancelling Wirel...,13283844475071838688,41.99,80.0,3900.0,4.6,soundcore US,True,47% OFF,all_products,All products,99,None,2
2,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,Bluetooth Headphones,3568096399987330642,17.99,160.0,73.0,4.0,Walmart - SISROW,False,88% OFF,all_products,All products,99,None,3
3,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,Sony Truly Wireless Earbuds WF-C510,13355876079085642476,48.00,70.0,4500.0,4.7,Best Buy,True,31% OFF,all_products,All products,99,None,4
4,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,JLab Audio Go Air True Wireless Earbuds Case,1396671360713973141,29.99,NaN,3900.0,3.8,Target,True,NaN,all_products,All products,99,None,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,Soundcore P40i by Anker Noise Cancelling Wirel...,13283844475071838688,41.99,80.0,3900.0,4.6,soundcore US,True,47% OFF,categorized_products,Deals on Wireless Earbuds,5,None,2
62,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,JBL Vibe Beam 2 True Wireless Noise Cancelling...,1749519698216963169,39.95,65.0,2200.0,4.3,Dell,True,38% OFF,categorized_products,Deals on Wireless Earbuds,5,None,3
63,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,Samsung Galaxy Buds3 Pro Wireless Earbuds,14563650187156512088,159.99,250.0,38000.0,4.3,Best Buy,True,36% OFF,categorized_products,Deals on Wireless Earbuds,5,None,4
64,146,wireless earbuds,1,2025-12-19 16:30:45.132971+00:00,JBL Tune Buds 2 Noise Cancelling True Wireless...,10005656883854915326,49.95,110.0,460.0,4.1,Sweetwater,True,54% OFF,categorized_products,Deals on Wireless Earbuds,5,None,5


## 8. Draft Staging Schema

The purpose of the staging layer (`stg_searches`, `stg_results`) is to
standardize and clean the raw JSON into a relational, analysis-ready format.

These tables:
- Remove presentation-layer noise from the API (thumbnails, snippet text, etc.)
- Normalize field names and data types
- Preserve the original search context
- Provide consistent module metadata for all result types
- Act as the foundation for downstream marts and dashboards

After prototyping the transformations above, the final schemas are:

---

### **8.1 `stg_searches` (one row per keyword search)**

| Column | Type | Description |
|--------|------|-------------|
| `raw_id` | INT | Primary key from the raw table (links back to raw JSON) |
| `pulled_at` | TIMESTAMP | When this keyword search was pulled |
| `keyword` | TEXT | Search query term |
| `page` | INT | Page number pulled (always 1 for now) |
| `location_used` | TEXT | city parameter used (hopefully `"Los Angeles"`) |
| `location_requested` | TEXT | city parameter requested (e.g., `"Los Angeles"`) |
| `gl` | TEXT | Country parameter (e.g., `"us"`) |
| `hl` | TEXT | Language parameter (e.g., `"en"`) |
| `device` | TEXT | Device type used in the search (`"desktop"`) |
| `num_results_requested` | INT | Requested SERP result count (e.g., 100) |
| `engine` | TEXT | Search engine used (`"google_shopping"`) |
| `google_domain` | TEXT | Google domain (`"google.com"`) |

---

### **8.2 `stg_results` (one row per product result)**

| Column | Type | Description |
|--------|------|-------------|
| `raw_id` | INT | Foreign key to `stg_searches` (search context) |
| `keyword` | TEXT | Keyword used for the search |
| `page` | INT | Page number of the search |
| `pulled_at` | TIMESTAMP | When the search was performed |
| `title` | TEXT | Product title |
| `product_id` | TEXT | SerpAPI product identifier |
| `price` | FLOAT | Extracted numeric price |
| `old_price` | FLOAT | Extracted numeric old price (if present) |
| `reviews` | INT | Number of reviews |
| `rating` | FLOAT | Star rating |
| `source` | TEXT | Seller / retailer |
| `multiple_sources` | BOOLEAN | Whether the product is offered by multiple sellers |
| `tag` | TEXT | Discount / badge text (e.g., `"40% OFF"`) |
| `module_type` | TEXT | Module category (`all_products`, `categorized_products`) |
| `module_label` | TEXT | Module title (subcategory name) |
| `module_index` | INT | Ordering of modules (99 = uncategorized, 100 = inline, 0–4 for categories) |
| `block_position` | TEXT | Placement of the inline module on the page (e.g., top; null for non-inline results) |
| `position_in_module` | INT | Position of the item inside its module |

---

### Notes

- `stg_results` may contain roughly **50–100 rows per keyword search**.
- `multiple_sources` is normalized to a boolean.
- `module_type`, `module_label`, and `module_index` unify both uncategorized results and category-specific results.
- These schemas match the output of the helper functions in Section 7 and will be used when writing the SQL `CREATE TABLE` scripts in the next notebook.

## 9. Conclusion

This notebook defined the staging layer structure and prototyped the transformations for `stg_searches` and `stg_results`.

### Next Steps
1. Write the SQL DDL script that creates both staging tables.  
2. Build the production staging notebook to:
   - Loop through all raw records that haven't yet been staged
   - Apply the transformation functions  
   - Load data into staging tables 
   
These steps complete the transition from exploration to the development of the formal staging pipeline.